## Causality

We would like to represent an acyclic SCM $M = (U, V, f_V, P_U)$ as a SOGA Probabilistic Program. We set the following assumptions:

- Exogenous variables $u \in U$ are independent from any other variable (exogenous or endogenous)
- All the varibles are declared only once (so we guarantee acyclicity). This in SOGA means in one block of instructions all x = something; x = x...
- Each endogenous variable depends only from one exogenous variables (to guarantee Markovianity)
- As this is a program, variables that are not yet declared can't be used
- If: non puoi definire solo variabili in un branch

Moreover, we need to:

- Be able to do interventions (modifying the lines of the code corresponding to the intervened variables), maybe directly from the distribution 
- Evaluate the program (likelihood + interventions or some independence test on the noise? )



In [24]:
from sogaPreprocessor import *
from producecfg import *
from smoothcfg import *
from libSOGA import *
from time import time

torch.set_default_dtype(torch.float64)

In [25]:
compiledFile=compile2SOGA('../programs/SOGA/Causality/SCM_1.soga')
cfg = produce_cfg(compiledFile)
smooth_cfg(cfg)

output_dist = start_SOGA(cfg)

In [26]:
output_dist_int =output_dist
output_dist_int

Dist<['a', 'b', 'd', 'c'],pi: tensor([[1.]])
mu: tensor([[2., 4., 2., 8.]])
sigma: tensor([[[ 4.0000,  8.0000,  4.0000, 12.0000],
         [ 8.0000, 17.0000,  8.0000, 25.0000],
         [ 4.0000,  8.0000,  5.0000, 13.0000],
         [12.0000, 25.0000, 13.0000, 39.0000]]])>

In [27]:
output_dist_int.var_list.index('a')

output_dist_int.gm.sigma[0][output_dist_int.var_list.index('a')][
    output_dist_int.var_list.index('b')] = 0

output_dist_int.gm.sigma[0][output_dist_int.var_list.index('b')][
    output_dist_int.var_list.index('a')] = 0

output_dist_int

Dist<['a', 'b', 'd', 'c'],pi: tensor([[1.]])
mu: tensor([[2., 4., 2., 8.]])
sigma: tensor([[[ 4.0000,  0.0000,  4.0000, 12.0000],
         [ 0.0000, 17.0000,  8.0000, 25.0000],
         [ 4.0000,  8.0000,  5.0000, 13.0000],
         [12.0000, 25.0000, 13.0000, 39.0000]]])>

In [28]:
output_dist_int.gm.mu

tensor([[2., 4., 2., 8.]])

In [29]:
import numpy as np

mu = output_dist_int.gm.mu[0].detach().numpy()
sigma = output_dist_int.gm.sigma[0].detach().numpy()
idx_b = output_dist_int.var_list.index('b')
idx_all = list(range(len(mu)))
idx_x1 = [i for i in idx_all if i != idx_b]
idx_x2 = [idx_b]

mu1 = mu[idx_x1]
mu2 = mu[idx_x2]
sigma11 = sigma[np.ix_(idx_x1, idx_x1)]
sigma12 = sigma[np.ix_(idx_x1, idx_x2)]
sigma21 = sigma[np.ix_(idx_x2, idx_x1)]
sigma22 = sigma[np.ix_(idx_x2, idx_x2)]

b_value = 3.0
mu_cond = mu1 + sigma12 @ np.linalg.inv(sigma22) @ (b_value - mu2)
sigma_cond = sigma11 - sigma12 @ np.linalg.inv(sigma22) @ sigma21

print("Conditional Mean:", mu_cond)
print("Conditional Covariance:", sigma_cond)

Conditional Mean: [2.         1.52941179 6.52941179]
Conditional Covariance: [[ 4.          4.         12.        ]
 [ 4.          1.23529434  1.23529434]
 [12.          1.23529434  2.23529534]]


In [31]:
compiledFile=compile2SOGA('../programs/SOGA/Causality/SCM_1_int.soga')
cfg = produce_cfg(compiledFile)
smooth_cfg(cfg)

output_dist = start_SOGA(cfg)
output_dist

Dist<['a', 'b', 'd', 'c'],pi: tensor([[1.]])
mu: tensor([[2., 3., 2., 7.]])
sigma: tensor([[[4.0000e+00, 0.0000e+00, 4.0000e+00, 4.0000e+00],
         [0.0000e+00, 1.0000e-06, 0.0000e+00, 1.0000e-06],
         [4.0000e+00, 0.0000e+00, 5.0000e+00, 5.0000e+00],
         [4.0000e+00, 1.0000e-06, 5.0000e+00, 6.0000e+00]]])>